# Miniproject 2

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
import sys
import math
import numpy as np
from collections import namedtuple

Pose = namedtuple("Pose", ["x", "y", "theta"])
Landmark = namedtuple("Landmark", ["x", "y"])
Measurement = namedtuple("Measurement", ["r", "b"])

def eval_gaussian(x, mean=0, var=1):
  """Return the Gaussian pdf"""
  return 1.0/math.sqrt(2*math.pi*var)*math.exp(-0.5*(x-mean)**2/var)

class Simulator:
  def __init__(self):
    self.robot = Pose(0, 0, 0)

    self.motion_noise_covariance = [[1, 0], [0, 1]]
    self.sensor_noise_covariance = [[1, 0], [0, np.deg2rad(5)**2]]

    self.landmarks = [Landmark(-1, -1), Landmark(-1, -1), Landmark(11, 11), Landmark(-1, 11)]

  def reset(self):
    self.robot = Pose(0, 0, 0)

  def simulate_motion(self, delta_p, delta_theta):
    noise = np.random.multivariate_normal([0, 0], self.motion_noise_covariance)
    x = self.robot.x + math.cos(self.robot.theta)*(delta_p + noise[0])
    y = self.robot.y + math.sin(self.robot.theta)*(delta_p + noise[0])
    theta = self.robot.theta + delta_theta + noise[1]
    self.robot = Pose(x, y, theta)

  def simulate_sensing(self):
    measurements = []
    for landmark in self.landmarks:
      noise = np.random.multivariate_normal([0, 0], self.sensor_noise_covariance)

      r = math.hypot(landmark[0]-self.robot.x, landmark[1]-self.robot.y) + noise[0]
      b = math.atan2(landmark[1]-self.robot.y, landmark[0]-self.robot.x) + noise[1]

      measurements.append(Measurement(r, b))

    return measurements




def drive_in_a_square(simulator):
  poses = []
  poses.append(simulator.robot)
  for sides in range(4):
    for i in range(100):
      simulator.simulate_motion(.1, 0)
      poses.append(simulator.robot)
    simulator.simulate_motion(0, np.pi/2)
    poses.append(simulator.robot)

  return np.array(poses)

def press(event):
  import matplotlib.pyplot as plt
  if event.key == 'q':
    sys.exit(0)
  plt.close()

def plot_samples(samples):
  import matplotlib.pyplot as plt
  fig, ax = plt.subplots()
  ax.axis([-2, 12, -2, 12])
  fig.canvas.mpl_connect('key_press_event', press)
  ax.plot(samples[:, 0], samples[:, 1], 'g+')
  plt.show()

def plot_trajectories(true_poses, estimated_poses):
  import matplotlib.pyplot as plt
  fig, ax = plt.subplots()
  ax.axis([-20, 20, -20, 20])
  fig.canvas.mpl_connect('key_press_event', press)

  ax.plot(true_poses[:, 0], true_poses[:,1], 'b')
  ax.plot(estimated_poses[:, 0], estimated_poses[:,1], 'r')
  plt.show()




## Problems

### Warmup 1
Let's make sure we know how to resample a set of
weights.

Please write a function that takes in a list of samples and associated weights,
and re-samples from the list of samples with replacement according the
likelihood of each sample given by the list of weights. Please generate 100 samples.

This should be an easy question --- you should feel free to use the sampling
helper functions in numpy. Specifically, take a look at the function numpy.random.choice.


For reference, our solution is **1** lines of code.

In [ ]:
def warmup_1(samples, weights):
raise NotImplementedError("Implement me!")

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(warmup_1(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=np.int64), np.array([0.00098125, 0.1012086 , 0.05777333, 0.11680514, 0.08591397, 0.11155254, 0.08240431, 0.1491899 , 0.17830923, 0.11586173], dtype=np.float64)), np.array([6, 8, 7, 6, 5, 7, 5, 9, 9, 5, 8, 6, 7, 9, 1, 1, 1, 8, 8, 8, 9, 8, 5, 8, 2, 7, 2, 9, 6, 5, 3, 8, 5, 7, 1, 7, 7, 7, 9, 7, 4, 5, 7, 1, 7, 7, 3, 2, 4, 5, 7, 5, 9, 1, 3, 3, 7, 3, 5, 3, 2, 2, 7, 2, 3, 5, 8, 1, 8, 1, 9, 5, 9, 7, 8, 1, 4, 2, 4, 2, 4, 5, 1, 7, 7, 3, 6, 1, 7, 9, 4, 7, 2, 8, 4, 3, 7, 1, 8, 1], dtype=np.int64), atol=1e-6)
print('Tests passed.')

### Warmup 2
We are going to be sampling from Gaussian
systems in this miniproject. Let us check that we know how to sample from a
multi-variate Gaussian.

Please write a function that takes in a mean vector, a covariance matrix and a
integer number of samples and returns a list of the requested number of
samples drawn from a Gaussian with the given mean and covariance.

This should be an easy question --- you should feel free to use the sampling
helper functions in numpy. Specifically, take a look at the function numpy.random.multivariate_normal.


For reference, our solution is **2** lines of code.

In [ ]:
def warmup_2(mean, covariance, num_samples):
  """Sample from a multi-variate Gaussian distribution.

  Args:
    mean: a vector in numpy.
    covariance: a matrix in numpy.
    num_samples: an integer.

  Returns:
    sample: a numpy vector with length num_samples.
  """
raise NotImplementedError("Implement me!")

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(warmup_2(np.array([ 0.35563307, -0.45998405,  0.47038804], dtype=np.float64), np.array([[2.18405112, 1.00414281, 1.49286069],  [1.00414281, 0.58927147, 0.64943064],  [1.49286069, 0.64943064, 1.09813033]], dtype=np.float64), 5), np.array([[-2.35284352, -1.74151879, -1.13022555],  [-2.79906741, -2.68743248, -1.56968391],  [-1.03394876, -1.07404052, -0.54291808],  [-0.41991666, -0.6895003 ,  0.26060185],  [-0.81622825, -0.99467384, -0.2275568 ]], dtype=np.float64), atol=1e-6)
print('Tests passed.')

### Sampling Question 1
Let us now start by building a motion model for a
quasi-holonomic ground robot.

A common motion model for such a robot assumes that the robot has a pose given
by $[x_t, y_t, \theta_t]$ at time $t$. The robot takes a command $[\Delta p_t,
\Delta \theta_t]$ which modifies the robot's pose accoding to the following:

$$
\begin{bmatrix} x_{t+1} \\ y_{t+1} \\ \theta_{t+1} \end{bmatrix} =
\begin{bmatrix} x_t + \cos(\theta_t) (\Delta p_t + \omega_t) \\
y_t + \sin(\theta_t)(\Delta p_t + \omega_t) \\
\theta_t + \Delta \theta_t + \nu_t
\end{bmatrix}
$$

where $[\omega_t, \nu_t]$ is Gaussian error injected at each time
step, distributed as $\omega_t \sim \mathcal{N}(0, \sigma^2_p)$, and
$\nu_t \sim \mathcal{N}(0, \sigma^2_\theta)$. The units of $\Delta  p_t$ is in
metres and $\Delta \theta_t$ in radians.

Let us assume that the robot's pose at each time step is a random variable
given as a set of samples.

Please implement a function that takes in a set of sampled poses
drawn from prior pose random variable $\mathbf{X}_t$
and a control $[\Delta p_t, \Delta \theta_t]$ and returns a set
of samples drawn from the posterior $X_{t+1}$.

At each step, please use numpy.random.multivariate_normal to sample the noise.


For reference, our solution is **8** lines of code.

In [ ]:
def motion_model(samples, delta_p, delta_theta, simulator):
  """A motion model (one-step forwarding).

  Args:
    samples: a list of samples.
    delta_p: translation command.
    delta_theta: rotation command.
    simulator: a Simulator object.

  Returns:
    a list of new samples by applying the motion model, as a single numpy array.
  """
new_samples = list()
for sample in samples:
    noise = np.random.multivariate_normal([0, 0], simulator.motion_noise_covariance)
return np.array(new_samples)

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(motion_model(np.array([[ 0.14940791,  0.03336743, -0.01790344],  [-0.08540957,  0.03130677, -0.22279039],  [ 0.08644362,  0.06536186, -0.06476612],  [-0.14543657,  0.22697546,  0.00399318],  [ 0.15327792, -0.01871839,  0.12822574],  [ 0.03781625,  0.01549474, -0.07747392],  [-0.03479121, -0.19807965,  0.01364402],  [ 0.12023798,  0.12302907, -0.03380064],  [-0.1048553 , -0.03023028, -0.12391994],  [ 0.19507754, -0.17062702, -0.04447554]], dtype=np.float64), 1, 0, Simulator()), np.array([[ 2.91301728, -0.01611596,  0.38225377],  [ 1.84442332, -0.40589916,  2.01810281],  [ 2.94798951, -0.12022892, -1.042044  ],  [ 1.8046363 ,  0.2347625 , -0.14736402],  [ 1.0426968 ,  0.09595719,  0.53882424],  [ 1.17842815, -0.07305016,  1.37679959],  [ 1.7260826 , -0.17405276,  0.13531904],  [ 1.5632765 ,  0.07423486,  0.29987369],  [ 2.37009854, -0.33850601, -0.32907821],  [ 1.50684678, -0.22900717, -0.89857128]], dtype=np.float64), atol=1e-6)
print('Tests passed.')

### Sampling Question 2
Let us now add in a measurement model. Let us assume that
there are four landmarks located at locations (-1, -1), (11, -1), (11, 11) and
(-1, 11). The robot is equipped with a sensor that can measure the range and
bearing [r, b] from the robot's location to each of these landmarks. Each of
these measurements is corrupted with noise $[q_r, q_b]$ distributed according
to $q_r \sim \mathcal{N}(0, \sigma^2_r)$ and $q_b \sim \mathcal{N}(0,
\sigma^2_b)$.

Given a set of measurements, we can use this model to compute the importance
weights of a set of samples of the robot pose.

Please implement a function that takes a set of samples and measurements and
returns the importance weights of the samples.


For reference, our solution is **16** lines of code.

In [ ]:
def compute_weights(samples, measurement, simulator):
  """Compute the importance weights of the samples.

  Args:
    samples: a list of samples.
    measurement: a measurement vector
    simulator: a Simulator object

  Returns:
    a list of importance weights, as a single numpy array.
  """
raise NotImplementedError("Implement me!")

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(compute_weights(np.array([[ 0.06536186, -0.25529898,  0.07543629],  [ 0.22697546, -0.0742165 , -0.12691735],  [-0.01871839,  0.00457585,  0.13376022],  [ 0.01549474,  0.14693588,  0.03300091],  [-0.19807965, -0.08877857, -0.03036106],  [ 0.12302907,  0.0156349 ,  0.10492744],  [-0.03023028, -0.03873268, -0.09150351],  [-0.17062702, -0.14200179,  0.17023727],  [-0.04380743, -0.05096522, -0.10932702],  [-0.16138978,  0.07774904, -0.01856509]], dtype=np.float64), [Measurement(r=0.5187470011794195, b=-2.3224308778331726), Measurement(r=0.9034084248042221, b=-2.4592240845320847), Measurement(r=15.528166957765391, b=0.8227771705573576), Measurement(r=11.111878239570428, b=1.6878518665472155)], Simulator()), np.array([0.01711671, 0.04172281, 0.14431346, 0.04618495, 0.05960219, 0.16749519, 0.17919444, 0.15658841, 0.18128191, 0.00649993], dtype=np.float64), atol=1e-6)
print('Tests passed.')

### Main Question
Now let's put everything together. Please write a method
localization() that does the following:

  * Initialize a set of samples drawn from a Gaussian distribution with mean
equal to the robot starting position, and a diagonal covariance matrix equal
to diag(1, 1, np.deg2rad(5))^2.
  * Drive the robot along the same pattern as in the drive_in_a_square method,
but on each step, after the command motion do the following:
  * use the motion_model method to update the sample between each simulated
motion,
  * use compute_weights to compute the sample weights from the
measurements,
  * resample a new set of samples,
  * blur the samples by applying noise drawn from a zero-mean Gaussian with
covariance equal to `diag(.001, .001, np.deg2rad(1))^2`.


For reference, our solution is **35** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `compute_weights`, `motion_model`. You may not need to use all of them.

In [ ]:
def localization(simulator):
  """Main function for robot localization.

  Args:
    simulator: a Simulator object.

  Returns:
    true_poses
    estimated_poses
  """
num_samples = 10
mean = simulator.robot
covariance = [[0.1, 0, 0], [0, 0.1, 0], [0, 0, (np.deg2rad(5) ** 2)]]
samples = np.random.multivariate_normal(mean, covariance, num_samples)
true_poses = []
true_poses.append(simulator.robot)
estimated_poses = []
estimated_poses.append(mean)
def resample(samples, weights):
    new_samples = np.zeros_like(samples)
    return new_samples
for sides in range(4):
    for i in range(10):
        (delta_p, delta_theta) = (1, 0)
    (delta_p, delta_theta) = (0, np.deg2rad(90))
return (np.array(true_poses), np.array(estimated_poses))

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
ret = localization(Simulator())
target = (np.array([[ 0.        ,  0.        ,  0.        ],  [ 0.25494743,  0.        ,  0.37816252],  [ 2.09797787,  0.73220589, -0.96959654],  [ 1.4613952 ,  1.66030481, -0.12523357],  [ 0.14770364,  1.82568859, -0.61855345],  [-0.0962629 ,  1.99932628, -0.24849756],  [-0.18413179,  2.02162232, -0.64334707],  [ 0.55322194,  1.46878446, -0.63805443],  [ 0.35445569,  1.61617443, -1.21932291],  [ 0.54511257,  1.09624717, -3.04906332],  [-0.37127229,  1.01121186, -1.68911477],  [-0.48181276,  0.08131326, -0.15341362],  [-0.74375495,  0.12181703, -1.94474117],  [-1.13554208, -0.87659965,  0.18912707],  [-0.02526097, -0.66407544, -1.85819654],  [ 0.04889197, -0.4132059 , -0.7936114 ],  [ 0.00960379, -0.373267  ,  0.25548179],  [ 1.38773021, -0.01331504,  0.08328099],  [ 1.73751182,  0.01588265,  1.13803892],  [ 1.63657132, -0.20262094, -0.12513399],  [ 1.41339514, -0.17454733, -0.83644035],  [ 1.75789252, -0.55613805, -1.34367482],  [ 2.15448367, -2.27217239,  0.98121717],  [ 2.2253805 , -2.16618954,  3.21323356],  [ 1.88505177, -2.1906128 ,  4.34101719],  [ 1.73295619, -2.58116007,  4.64828197],  [ 1.78876609, -1.71177892,  6.07459884],  [ 2.68684617, -1.90187117,  5.79568456],  [ 3.41102817, -2.28581952,  5.54726525],  [ 4.12192911, -2.92963681,  4.73763868],  [ 4.12241356, -2.94881908,  4.03377964],  [ 3.74465719, -3.41728794,  3.97768017],  [ 3.86464129, -3.28447893,  4.50751395],  [ 3.71127123, -4.02257858,  5.19012207],  [ 3.95013141, -4.48393649,  6.11455609],  [ 2.52830103, -4.24187558,  6.97670415],  [ 3.0923896 , -3.77296615,  6.36788903],  [ 2.77828838, -3.7996355 ,  5.69422486],  [ 4.68131737, -5.07095612,  5.47445187],  [ 4.52182413, -4.90383998,  5.9247407 ],  [ 5.23539919, -5.17116527,  5.58386629],  [ 5.43375365, -5.33800615,  4.5957696 ],  [ 5.55010547, -4.34482748,  4.19141738],  [ 4.84049773, -5.58139532,  3.84943511],  [ 5.91239398, -4.66408357,  5.58514416]], dtype=np.float64), np.array([[  0.        ,   0.        ,   0.        ],  [  0.19058789,   0.08076551,   0.13015944],  [  1.73884213,   0.77528172,   1.13724432],  [  2.32272768,   1.53897586,  -0.20335719],  [  0.91254162,   2.07992579,   0.33041101],  [ -0.10249755,   1.77076375,  -0.15929873],  [ -0.053636  ,   1.7357245 ,  -1.24775663],  [  0.33362574,   2.15292384,   0.76494676],  [ -0.06330826,   1.0094867 ,  -0.41082755],  [  0.42589094,   0.79559174,  -0.0257862 ],  [ -0.29166286,   0.75716514,  -1.2357107 ],  [ -0.20078089,   0.51236446,  -0.07605858],  [ -0.54218515,   0.51447095,  -0.82622069],  [ -1.45113278,   1.67386395,  -1.31883524],  [ -0.98375982,   0.0144982 ,  -1.5884315 ],  [ -0.94672607,  -0.58853722,  -3.41545344],  [ -0.68729134,  -0.70101432,  -3.73389319],  [ -0.52032134,  -0.79368716,  -5.74997517],  [  0.95875632,   0.06233885,  -5.11169347],  [  0.74297861,  -0.53299447,  -6.46674077],  [  0.34326157,  -0.50194941,  -7.40239332],  [  0.2678123 ,  -0.46473962,  -8.8903894 ],  [ -0.91195431,  -1.16980026,  -6.41626306],  [  0.83093655,  -1.39284578,  -6.69312925],  [  1.86644767,  -1.7932945 ,  -5.37147095],  [  1.98493433,  -2.62941749,  -4.46135281],  [  1.61630803,  -1.30743702,  -5.18185803],  [  1.59268985,  -1.45690111,  -5.5066592 ],  [  2.01038262,  -1.81736801,  -7.06878497],  [  2.87208808,  -2.67657584,  -8.30025262],  [  2.9156086 ,  -2.60625252,  -7.89019499],  [  3.14312724,  -3.12334753,  -8.6267845 ],  [  3.00649234,  -3.2606006 , -10.16542017],  [  3.28547325,  -3.19132734,  -9.12541345],  [  3.83713075,  -3.56883755,  -9.00432044],  [  2.8975788 ,  -4.54553428,  -9.16020851],  [  3.6258195 ,  -4.68640943, -10.57959091],  [  3.10800677,  -3.60614824,  -9.75828815],  [  3.50262302,  -3.71522404, -10.62331535],  [  3.96464251,  -4.26933193,  -9.23618253],  [  4.25193587,  -4.25450217,  -9.33476329],  [  3.71038933,  -4.58405451,  -8.7792178 ],  [  4.58125857,  -4.53201125,  -9.68885746],  [  5.11925005,  -5.17502524, -10.90162278],  [  5.33818285,  -4.44455145,  -9.17833356]], dtype=np.float64))
assert all((np.allclose(x, y, atol=1e-6) for x, y in zip(ret, target)))
print('Tests passed.')